In [ ]:
%matplotlib inline

# Limit state based on resampling.


In [ ]:
from __future__ import annotations

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from gemseo.algos.design_space import DesignSpace
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.mlearning.regression.algos.rbf import RBFRegressor
from numpy import array
from numpy import linspace
from numpy import zeros

from gemseo_mlearning import sample_discipline
from gemseo_mlearning.active_learning.acquisition_criteria.quantile import Quantile
from gemseo_mlearning.active_learning.active_learning_algo import ActiveLearningAlgo
from gemseo_mlearning.active_learning.distributions.regressor_distribution import (
    RegressorDistribution,
)

l_b = -2
u_b = 2

n_test = 20
level = 0.8

Definition of the discipline
----------------------------



In [ ]:
discipline = AnalyticDiscipline({"z": "(1-x)**2+100*(y-x**2)**2"}, name="Rosenbrock")

Definition of the input space
-----------------------------



In [ ]:
input_space = DesignSpace()
input_space.add_variable("x", l_b=-2, u_b=2, value=1.0)
input_space.add_variable("y", l_b=-2, u_b=2, value=1.0)

Initial surrogate model
-----------------------



In [ ]:
learning_dataset = sample_discipline(discipline, input_space, "z", "OT_OPT_LHS", 30)
algo = RBFRegressor(learning_dataset)

Universal distribution for this surrogate model
-----------------------------------------------



In [ ]:
distribution = RegressorDistribution(algo, bootstrap=False)
distribution.learn()

Data acquisition to improve the surrogate model
-----------------------------------------------



In [ ]:
acquisition = ActiveLearningAlgo("Quantile", input_space, distribution, level=level)
acquisition.set_acquisition_algorithm("fullfact", n_samples=1000)
acquisition.update_algo(discipline, 20)

points = distribution.algo.learning_set[["x", "y"]]
points_x = points["x"]
points_y = points["y"]

Evaluation of discipline and expected improvement
-------------------------------------------------



In [ ]:
crit = Quantile(distribution, level)
x_test = linspace(l_b, u_b, n_test)
disc_data = zeros((n_test, n_test))
crit_data = zeros((n_test, n_test))
surr_data = zeros((n_test, n_test))
for i in range(n_test):
    for j in range(n_test):
        xij = array([x_test[j], x_test[i]])
        input_data = {"x": array([xij[0]]), "y": array([xij[1]])}
        disc_data[i, j] = discipline.execute(input_data)["z"][0]
        crit_data[i, j] = crit(xij)
        surr_data[i, j] = algo.predict(xij)[0]

Plotting
--------



In [ ]:
train = learning_dataset.get_view(variable_names=["x", "y"]).to_numpy()
x_train = train["x"]
y_train = train["y"]
fig = plt.figure(constrained_layout=True)
spec = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
axes = [[None, None], [None, None]]
titles = [["Discipline", "Infill criterion"], ["Surrogate model", None]]
data = [[disc_data, crit_data], [surr_data, None]]
for i in range(2):
    for j in range(2):
        if [i, j] != [1, 1]:
            axes[i][j] = fig.add_subplot(spec[i, j])
            axes[i][j].contourf(x_test, x_test, data[i][j])
            axes[i][j].plot(x_train[:, 0], y_train[:, 0], "w+", ms=1)
            for index, _ in enumerate(points_x):
                axes[i][j].plot(points_x[index, 0], points_y[index, 0], "wo", ms=1)
                axes[i][j].annotate(
                    index + 1,
                    (points_x[index, 0] + 0.05, points_y[index, 0] + 0.05),
                    color="white",
                )
            axes[i][j].set_title(titles[i][j])
plt.show()